In [97]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

## Cargar datos


In [98]:
transacciones = pd.read_csv('../Datos/transacciones_con_features.csv', encoding='utf-8')
cotizaciones = pd.read_csv('../Datos/cotizaciones_con_features.csv', encoding='utf-8')

print(f"Transacciones: {transacciones.shape}")
print(f"Cotizaciones: {cotizaciones.shape}")


Transacciones: (2099287, 43)
Cotizaciones: (178378, 22)


## Convertir fechas

In [99]:
transacciones['fecha'] = pd.to_datetime(transacciones['fecha'])

print(f"Rango transacciones: {transacciones['fecha'].min()} a {transacciones['fecha'].max()}")


Rango transacciones: 1971-01-02 00:00:00 a 1973-01-31 00:00:00


# Dividir transacciones



## Analisis de Overlap

In [100]:
train_data = pd.read_csv('../Datos/validacion_historica/transacciones_entrenamiento.csv', encoding='utf-8')
test_data = pd.read_csv('../Datos/validacion_historica/transacciones_validacion.csv', encoding='utf-8')

In [101]:
clientes_entrenamiento = set(transacciones['id'].unique())
clientes_validacion = set(test_data['id'].unique())
clientes_comunes = clientes_entrenamiento.intersection(clientes_validacion)

productos_entrenamiento = set(train_data['producto'].unique())
productos_validacion = set(test_data['producto'].unique())
productos_comunes = productos_entrenamiento.intersection(productos_validacion)

print(f"Clientes comunes: {len(clientes_comunes):,} / {len(clientes_entrenamiento):,} ({len(clientes_comunes)/len(clientes_entrenamiento)*100:.1f}%)")
print(f"Productos comunes: {len(productos_comunes):,} / {len(productos_entrenamiento):,} ({len(productos_comunes)/len(productos_entrenamiento)*100:.1f}%)")


Clientes comunes: 25,950 / 419,221 (6.2%)
Productos comunes: 3,085 / 7,177 (43.0%)


## Importar Modelos

In [102]:
import os
import joblib
import pandas as pd
import numpy as np
from collections import defaultdict

# Obtener la ruta del directorio actual (Modelos)
MODELOS_DIR = "../Modelos/ModelosTrain"

# Cargar los modelos entrenados desde archivos .pkl
print("Cargando modelos entrenados...")

try:
    content_model = joblib.load(os.path.join(MODELOS_DIR, 'content_model_train.pkl'))
    print("✓ Modelo de contenido cargado")
except Exception as e:
    print(f"⚠️ Error cargando modelo de contenido: {e}")
    content_model = None

try:
    copurchase_model = joblib.load(os.path.join(MODELOS_DIR, 'co_purchase_model_train.pkl'))
    print("✓ Modelo de co-compra cargado")
except Exception as e:
    print(f"⚠️ Error cargando modelo de co-compra: {e}")
    copurchase_model = None

try:
    coquote_model = joblib.load(os.path.join(MODELOS_DIR, 'co_quotation_model_train.pkl'))
    print("✓ Modelo de co-cotización cargado")
except Exception as e:
    print(f"⚠️ Error cargando modelo de co-cotización: {e}")
    coquote_model = None

try:
    popularity_model = joblib.load(os.path.join(MODELOS_DIR, 'popularity_model_train.pkl'))
    print("✓ Modelo de popularidad cargado")
except Exception as e:
    print(f"⚠️ Error cargando modelo de popularidad: {e}")
    popularity_model = None

try:
    hybrid_model = joblib.load(os.path.join(MODELOS_DIR, 'hybrid_model_train.pkl'))
    print("✓ Modelo híbrido cargado")
except Exception as e:
    print(f"⚠️ Error cargando modelo híbrido: {e}")
    hybrid_model = None

print("Todos los modelos han sido cargados.\n")

# Mostrar información de los modelos cargados
print("📊 Información de los modelos:")
if content_model:
    print(f"  - Contenido: {content_model['similarity_matrix'].shape[0]:,} productos")
if copurchase_model:
    co_matrix = copurchase_model['co_occurrence_matrix']
    print(f"  - Co-compra: {co_matrix.shape[0]:,} productos, densidad: {co_matrix.nnz / (co_matrix.shape[0] * co_matrix.shape[1]):.6f}")
if coquote_model:
    cq_matrix = coquote_model['co_quotation_matrix']
    print(f"  - Co-cotización: {cq_matrix.shape[0]:,} productos, densidad: {cq_matrix.nnz / (cq_matrix.shape[0] * cq_matrix.shape[1]):.6f}")
if popularity_model:
    print(f"  - Popularidad: {len(popularity_model['available_metrics'])} métricas, {len(popularity_model['popularity_rankings'])} rankings")
if hybrid_model:
    weights = hybrid_model['default_weights']
    print(f"  - Híbrido: 4 métodos con pesos {weights}")

Cargando modelos entrenados...
✓ Modelo de contenido cargado
✓ Modelo de co-compra cargado
✓ Modelo de co-cotización cargado
✓ Modelo de popularidad cargado
✓ Modelo híbrido cargado
Todos los modelos han sido cargados.

📊 Información de los modelos:
  - Contenido: 7,221 productos
  - Co-compra: 7,221 productos, densidad: 0.025437
  - Co-cotización: 7,221 productos, densidad: 0.002669
  - Popularidad: 4 métricas, 16 rankings
  - Híbrido: 4 métodos con pesos {'content_weight': 0.25, 'cf_buy_weight': 0.4, 'cf_quote_weight': 0.15, 'popularity_weight': 0.2}


## Funciones de Recomendacion
Aca se cargan las funciones de recomendacion de cada modelo por individual y el hibrido que usa los 4 

In [103]:

def get_content_recommendations(input_product, N=10):
    """
    Genera recomendaciones basadas en contenido usando el modelo cargado.
    
    Args:
        input_product (str): ID del producto
        N (int): Número de recomendaciones
    
    Returns:
        pandas.DataFrame: Recomendaciones con columnas ['producto', 'similarity_score']
    """
    if content_model is None:
        print("Error: Modelo de contenido no disponible")
        return pd.DataFrame()
    
    similarity_matrix = content_model['similarity_matrix']
    product_to_idx = content_model['product_to_idx']
    idx_to_product = content_model['idx_to_product']
    
    if input_product not in product_to_idx:
        print(f"Error: Producto '{input_product}' no encontrado")
        return pd.DataFrame()
    
    idx = product_to_idx[input_product]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    recommendations = []
    for i, score in sim_scores:
        if i == idx:
            continue
        product_name = idx_to_product.get(i)
        if product_name and len(recommendations) < N:
            recommendations.append({'producto': product_name, 'similarity_score': score})
        elif len(recommendations) >= N:
            break
    
    return pd.DataFrame(recommendations)

def get_copurchase_recommendations(input_product, N=10):
    """
    Genera recomendaciones basadas en co-compra usando el modelo cargado.
    
    Args:
        input_product (str): ID del producto
        N (int): Número de recomendaciones
    
    Returns:
        pandas.DataFrame: Recomendaciones con columnas ['producto', 'co_purchase_count']
    """
    if copurchase_model is None:
        #print("Error: Modelo de co-compra no disponible")
        return pd.DataFrame()
    
    co_occurrence_matrix = copurchase_model['co_occurrence_matrix']
    product_to_idx = copurchase_model['product_to_idx']
    idx_to_product = copurchase_model['idx_to_product']
    
    if input_product not in product_to_idx:
        print(f"Error: Producto '{input_product}' no encontrado")
        return pd.DataFrame()
    
    idx = product_to_idx[input_product]
    co_buys = co_occurrence_matrix[idx, :]
    co_buy_indices = co_buys.indices
    co_buy_values = co_buys.data
    
    if len(co_buy_indices) == 0:
        #print(f"Producto '{input_product}' no tiene co-compras registradas")
        return pd.DataFrame()
    
    recommendations = []
    cobuy_pairs = sorted(zip(co_buy_indices, co_buy_values), key=lambda x: x[1], reverse=True)
    
    for i, count in cobuy_pairs:
        if i == idx:
            continue
        product_name = idx_to_product.get(i)
        if product_name and len(recommendations) < N:
            recommendations.append({'producto': product_name, 'co_purchase_count': count})
        elif len(recommendations) >= N:
            break
    
    return pd.DataFrame(recommendations)

def get_coquotation_recommendations(input_product, N=10):
    """
    Genera recomendaciones basadas en co-cotización usando el modelo cargado.
    
    Args:
        input_product (str): ID del producto
        N (int): Número de recomendaciones
    
    Returns:
        pandas.DataFrame: Recomendaciones con columnas ['producto', 'co_quotation_count']
    """
    if coquote_model is None:
        #print("Error: Modelo de co-cotización no disponible")
        return pd.DataFrame()
    
    co_quotation_matrix = coquote_model['co_quotation_matrix']
    product_to_idx = coquote_model['product_to_idx']
    idx_to_product = coquote_model['idx_to_product']
    
    if input_product not in product_to_idx:
        print(f"Error: Producto '{input_product}' no encontrado")
        return pd.DataFrame()
    
    idx = product_to_idx[input_product]
    co_quotes = co_quotation_matrix[idx, :]
    co_quote_indices = co_quotes.indices
    co_quote_values = co_quotes.data
    
    if len(co_quote_indices) == 0:
        None
        #print(f"Producto '{input_product}' no tiene co-cotizaciones registradas")
        return pd.DataFrame()
    
    recommendations = []
    coquote_pairs = sorted(zip(co_quote_indices, co_quote_values), key=lambda x: x[1], reverse=True)
    
    for i, count in coquote_pairs:
        if i == idx:
            continue
        product_name = idx_to_product.get(i)
        if product_name and len(recommendations) < N:
            recommendations.append({'producto': product_name, 'co_quotation_count': count})
        elif len(recommendations) >= N:
            break
    
    return pd.DataFrame(recommendations)

def get_popularity_recommendations(input_product_features, N=10, popularity_metric='total_unidades_vendidas'):
    """
    Genera recomendaciones basadas en popularidad para productos nuevos usando el modelo cargado.
    
    Args:
        input_product_features (dict): Características del producto con claves:
                                     ['subcategoria', 'categoria', 'categoria_macro']
        N (int): Número de recomendaciones
        popularity_metric (str): Métrica de popularidad a usar
    
    Returns:
        pandas.DataFrame: Recomendaciones con columnas ['producto', 'nivel_categoria', popularity_metric]
    """
    if popularity_model is None:
        return pd.DataFrame()
    
    product_features = popularity_model['product_features']
    popularity_rankings = popularity_model['popularity_rankings']
    
    if popularity_metric not in popularity_model['available_metrics']:
        return pd.DataFrame()
    
    recommendations = []
    remaining_slots = N
    
    # Extraer características categóricas
    subcategoria_nueva = input_product_features.get('subcategoria')
    categoria_nueva = input_product_features.get('categoria')
    categoria_macro_nueva = input_product_features.get('categoria_macro')

    
    # --- NIVEL 1: Misma Subcategoría ---
    if subcategoria_nueva and remaining_slots > 0:
        subcategoria_products = product_features[
            product_features['subcategoria'] == subcategoria_nueva
        ].sort_values(popularity_metric, ascending=False).head(remaining_slots)
        
        for producto in subcategoria_products.index:
            recommendations.append({
                'producto': producto,
                'nivel_categoria': 'subcategoria',
                popularity_metric: subcategoria_products.loc[producto, popularity_metric]
            })
        
        remaining_slots -= len(subcategoria_products)

    
    # --- NIVEL 2: Misma Categoría ---
    if categoria_nueva and remaining_slots > 0:
        already_recommended = [r['producto'] for r in recommendations]
        categoria_products = product_features[
            (product_features['categoria'] == categoria_nueva) &
            (~product_features.index.isin(already_recommended))
        ].sort_values(popularity_metric, ascending=False).head(remaining_slots)
        
        for producto in categoria_products.index:
            recommendations.append({
                'producto': producto,
                'nivel_categoria': 'categoria',
                popularity_metric: categoria_products.loc[producto, popularity_metric]
            })
        
        remaining_slots -= len(categoria_products)

    # --- NIVEL 3: Misma Categoría Macro ---
    if categoria_macro_nueva and remaining_slots > 0:
        already_recommended = [r['producto'] for r in recommendations]
        categoria_macro_products = product_features[
            (product_features['categoria_macro'] == categoria_macro_nueva) &
            (~product_features.index.isin(already_recommended))
        ].sort_values(popularity_metric, ascending=False).head(remaining_slots)
        
        for producto in categoria_macro_products.index:
            recommendations.append({
                'producto': producto,
                'nivel_categoria': 'categoria_macro',
                popularity_metric: categoria_macro_products.loc[producto, popularity_metric]
            })
        
        remaining_slots -= len(categoria_macro_products)

    # --- NIVEL 4: Más Populares en General ---
    if remaining_slots > 0:
        already_recommended = [r['producto'] for r in recommendations]
        general_products = product_features[
            ~product_features.index.isin(already_recommended)
        ].sort_values(popularity_metric, ascending=False).head(remaining_slots)
        
        for producto in general_products.index:
            recommendations.append({
                'producto': producto,
                'nivel_categoria': 'general',
                popularity_metric: general_products.loc[producto, popularity_metric]
            })
        

    # Convertir a DataFrame
    if recommendations:
        results_df = pd.DataFrame(recommendations)
        # Renombrar la columna de métrica para consistencia
        results_df = results_df.rename(columns={popularity_metric: 'popularity_score'})
        return results_df
    else:
        print("❌ No se pudieron generar recomendaciones.")
        return pd.DataFrame()
    
def get_hybrid_recommendations(input_product, N=10, k=2, 
                             content_weight=0.05, cf_buy_weight=0.8, 
                             cf_quote_weight=0.1, popularity_weight=0.05,
                             input_product_features=None):
    """
    Genera recomendaciones híbridas usando re-ranking con 4 métodos.
    
    Args:
        input_product (str): ID del producto
        N (int): Número de recomendaciones finales
        content_weight (float): Peso para contenido
        cf_buy_weight (float): Peso para co-compra
        cf_quote_weight (float): Peso para co-cotización
        popularity_weight (float): Peso para popularidad
        k (int): Constante de suavizado
        input_product_features (dict): Para productos nuevos
    
    Returns:
        pandas.DataFrame: Recomendaciones con columnas ['producto', 'hybrid_score']
    """
    if hybrid_model is None:
        return pd.DataFrame()
    
    # Verificar si es producto nuevo
    product_to_idx = hybrid_model.get('product_to_idx', {})
    is_new_product = input_product not in product_to_idx
    
    if is_new_product and input_product_features is None:
        print(f"Error: Producto '{input_product}' es nuevo pero no se proporcionaron características")
        return pd.DataFrame()
    
    # Lógica adaptativa para productos nuevos
    if is_new_product:
        print(f"🆕 Producto '{input_product}' es nuevo. Usando solo recomendaciones de popularidad.")
        content_weight = 0.0
        cf_buy_weight = 0.0
        cf_quote_weight = 0.0
        popularity_weight = 1.0
    
    # Obtener candidatos de cada método
    content_recs = get_content_recommendations(input_product, N=50) if not is_new_product else pd.DataFrame()
    copurchase_recs = get_copurchase_recommendations(input_product, N=50) if not is_new_product else pd.DataFrame()
    coquote_recs = get_coquotation_recommendations(input_product, N=50) if not is_new_product else pd.DataFrame()
    
    # Para popularidad, usar características apropiadas
    if is_new_product:
        popularity_recs = get_popularity_recommendations(input_product_features, N=50)
    else:
        # Para producto existente, obtener sus características del modelo
        if popularity_model and 'product_features' in popularity_model:
            product_features_df = popularity_model['product_features']
            if input_product in product_features_df.index:
                existing_features = {
                    'subcategoria': product_features_df.loc[input_product, 'subcategoria'],
                    'categoria': product_features_df.loc[input_product, 'categoria'],
                    'categoria_macro': product_features_df.loc[input_product, 'categoria_macro']
                }
                popularity_recs = get_popularity_recommendations(existing_features, N=50)
                # Excluir el producto de entrada
                popularity_recs = popularity_recs[popularity_recs['producto'] != input_product]
            else:
                popularity_recs = pd.DataFrame()
        else:
            popularity_recs = pd.DataFrame()
    
    # Lógica adaptativa para productos con pocas co-compras
    if not is_new_product:
        if copurchase_recs.empty and coquote_recs.empty:
            content_weight = 0.4
            cf_buy_weight = 0.0
            cf_quote_weight = 0.0
            popularity_weight = 0.6
        elif copurchase_recs.empty:
            content_weight = 0.2
            cf_buy_weight = 0.0
            cf_quote_weight = 0.5
            popularity_weight = 0.3
        elif len(copurchase_recs) > 0 and copurchase_recs.iloc[0]['co_purchase_count'] < 10:
            content_weight = 0.2
            cf_buy_weight = 0.2
            cf_quote_weight = 0.1
            popularity_weight = 0.5
    
    # Calcular scores híbridos
    hybrid_scores = defaultdict(float)
    
    # Contribución del modelo de contenido
    if not content_recs.empty and content_weight > 0:
        for rank, row in enumerate(content_recs.itertuples(), 1):
            score_contribution = content_weight * (1.0 / (rank + k))
            hybrid_scores[row.producto] += score_contribution
    
    # Contribución del modelo de co-compra
    if not copurchase_recs.empty and cf_buy_weight > 0:
        for rank, row in enumerate(copurchase_recs.itertuples(), 1):
            score_contribution = cf_buy_weight * (1.0 / (rank + k))
            hybrid_scores[row.producto] += score_contribution
    
    # Contribución del modelo de co-cotización
    if not coquote_recs.empty and cf_quote_weight > 0:
        for rank, row in enumerate(coquote_recs.itertuples(), 1):
            score_contribution = cf_quote_weight * (1.0 / (rank + k))
            hybrid_scores[row.producto] += score_contribution
    
    # Contribución del modelo de popularidad
    if not popularity_recs.empty and popularity_weight > 0:
        for rank, row in enumerate(popularity_recs.itertuples(), 1):
            score_contribution = popularity_weight * (1.0 / (rank + k))
            hybrid_scores[row.producto] += score_contribution
    
    if not hybrid_scores:
        print(f"No se encontraron candidatos para '{input_product}'")
        return pd.DataFrame()
    
    # Crear DataFrame final
    final_recs = pd.DataFrame(hybrid_scores.items(), columns=['producto', 'hybrid_score'])
    final_recs = final_recs[final_recs['producto'] != input_product]
    final_recs = final_recs.sort_values('hybrid_score', ascending=False).head(N)
    
    
    return final_recs


## Funcion para probar todos los modelos

In [104]:
def test_all_models(test_product, N=5, input_product_features=None):
    """
    Prueba todos los modelos con un producto de ejemplo y devuelve los resultados.
    
    Args:
        test_product (str): Producto para probar
        N (int): Número de recomendaciones por modelo
        input_product_features (dict): Para productos nuevos. Diccionario con
                                     ['subcategoria', 'categoria', 'categoria_macro']
    
    Returns:
        dict: Diccionario con las recomendaciones de los 5 modelos:
              {
                'content': DataFrame,
                'co_purchase': DataFrame, 
                'co_quotation': DataFrame,
                'popularity': DataFrame,
                'hybrid': DataFrame,
                'input_product': str,
                'is_new_product': bool
              }
    """
    
    # Verificar si es producto nuevo
    product_to_idx = content_model.get('product_to_idx', {}) if content_model else {}
    is_new_product = test_product not in product_to_idx
    
    # Inicializar diccionario de resultados
    results = {
        'input_product': test_product,
        'is_new_product': is_new_product,
        'content': pd.DataFrame(),
        'co_purchase': pd.DataFrame(),
        'co_quotation': pd.DataFrame(),
        'popularity': pd.DataFrame(),
        'hybrid': pd.DataFrame()
    }
    
    # Probar modelo de contenido (solo para productos existentes)
    if not is_new_product:
        content_recs = get_content_recommendations(test_product, N=N)
        results['content'] = content_recs
    
    # Probar modelo de co-compra (solo para productos existentes)
    if not is_new_product:
        copurchase_recs = get_copurchase_recommendations(test_product, N=N)
        results['co_purchase'] = copurchase_recs
    
    # Probar modelo de co-cotización (solo para productos existentes)
    if not is_new_product:
        coquote_recs = get_coquotation_recommendations(test_product, N=N)
        results['co_quotation'] = coquote_recs
    
    # Probar modelo de popularidad
    if is_new_product and input_product_features is not None:
        # Para producto nuevo, usar características proporcionadas
        popularity_recs = get_popularity_recommendations(input_product_features, N=N)
        results['popularity'] = popularity_recs
    elif not is_new_product and popularity_model:
        # Para producto existente, obtener características del modelo
        try:
            product_features_df = popularity_model['product_features']
            if test_product in product_features_df.index:
                existing_features = {
                    'subcategoria': product_features_df.loc[test_product, 'subcategoria'],
                    'categoria': product_features_df.loc[test_product, 'categoria'],
                    'categoria_macro': product_features_df.loc[test_product, 'categoria_macro']
                }
                popularity_recs = get_popularity_recommendations(existing_features, N=N)
                # Excluir el producto de entrada
                popularity_recs = popularity_recs[popularity_recs['producto'] != test_product]
                results['popularity'] = popularity_recs
        except Exception as e:
            pass  # Silencioso en caso de error
    
    # Probar modelo híbrido
    hybrid_recs = get_hybrid_recommendations(test_product, N=N, input_product_features=input_product_features)
    results['hybrid'] = hybrid_recs
    
    return results

# Función auxiliar para probar producto nuevo
def test_new_product_example():
    """Ejemplo de cómo probar un producto nuevo"""
    producto_nuevo_features = {
        'subcategoria': 'subcategoria_1',
        'categoria': 'categoria_1', 
        'categoria_macro': 'categoria_macro_1'
    }
    
    results = test_all_models(
        "producto_nuevo_123", 
        N=5, 
        input_product_features=producto_nuevo_features
    )
    
    return results

# Exportar todo
__all__ = [
    'content_model', 
    'copurchase_model', 
    'coquote_model', 
    'popularity_model',
    'hybrid_model',
    'get_content_recommendations',
    'get_copurchase_recommendations',
    'get_coquotation_recommendations',
    'get_popularity_recommendations',
    'get_hybrid_recommendations',
    'test_all_models',
    'test_new_product_example',
]

## Ejemplo de uso

In [105]:
producto = "producto_2"
recomendaciones = test_all_models(producto, N=5)
recomendaciones

{'input_product': 'producto_2',
 'is_new_product': False,
 'content':         producto  similarity_score
 0    producto_17          0.999949
 1   producto_652          0.999639
 2  producto_1232          0.999382
 3  producto_1476          0.999345
 4  producto_1944          0.999143,
 'co_purchase':         producto  co_purchase_count
 0   producto_413                 17
 1  producto_1120                 16
 2   producto_119                 16
 3   producto_110                 15
 4  producto_3126                 15,
 'co_quotation':         producto  co_quotation_count
 0  producto_3126                  13
 1  producto_1315                   6
 2   producto_208                   6
 3   producto_632                   6
 4   producto_642                   6,
 'popularity':         producto nivel_categoria  popularity_score
 0    producto_75    subcategoria            2719.0
 1   producto_201    subcategoria            2023.0
 2  producto_1165    subcategoria            2009.0
 3   prod

## Preparar Datos de validación


In [106]:
# Crear diccionario de compras reales en período de validación
compras_reales = defaultdict(set)
for _, row in test_data.iterrows():
    compras_reales[row['id']].add(row['producto'])

print(f"Clientes únicos en validación: {len(compras_reales)}")

# ================================================================
# PREPARAR DATOS DE INGRESOS PARA ANÁLISIS ECONÓMICO - MODELO HÍBRIDO
# ================================================================

print("PREPARANDO DATOS DE INGRESOS PARA MODELO HÍBRIDO")
print("="*50)

# Crear diccionarios de ingresos para evaluación por transacción/pedido
ingresos_por_pedido_producto = defaultdict(dict)
ingresos_totales_por_pedido = defaultdict(float)

# Crear diccionarios de ingresos por cliente
ingresos_por_cliente_producto = defaultdict(dict)
ingresos_totales_por_cliente = defaultdict(float)

for _, row in test_data.iterrows():
    pedido_id = row['pedido']
    cliente_id = row['id']
    producto = row['producto']
    precio = row['precio']
    cantidad = row['cantidad']
    ingreso_transaccion = precio * cantidad
    
    # INGRESOS POR PEDIDO (para evaluación de ventas cruzadas por transacción)
    if producto in ingresos_por_pedido_producto[pedido_id]:
        ingresos_por_pedido_producto[pedido_id][producto] += ingreso_transaccion
    else:
        ingresos_por_pedido_producto[pedido_id][producto] = ingreso_transaccion
    
    ingresos_totales_por_pedido[pedido_id] += ingreso_transaccion
    
    # INGRESOS POR CLIENTE (para evaluación personalizada)
    if producto in ingresos_por_cliente_producto[cliente_id]:
        ingresos_por_cliente_producto[cliente_id][producto] += ingreso_transaccion
    else:
        ingresos_por_cliente_producto[cliente_id][producto] = ingreso_transaccion
    
    ingresos_totales_por_cliente[cliente_id] += ingreso_transaccion

# Crear diccionario de precios promedio por producto
precios_promedio_productos = defaultdict(list)
for _, row in test_data.iterrows():
    precios_promedio_productos[row['producto']].append(row['precio'])

precio_promedio_por_producto = {}
for producto, precios in precios_promedio_productos.items():
    precio_promedio_por_producto[producto] = np.mean(precios)

# Estadísticas generales
ingresos_pedidos = list(ingresos_totales_por_pedido.values())
ingresos_clientes = list(ingresos_totales_por_cliente.values())

print(f"ESTADÍSTICAS DE INGRESOS:")
print(f"   📋 Por Pedido:")
print(f"      - Total pedidos únicos: {len(ingresos_totales_por_pedido):,}")
print(f"      - Ingreso promedio por pedido: ${np.mean(ingresos_pedidos):.2f}")
print(f"      - Ingreso mediano por pedido: ${np.median(ingresos_pedidos):.2f}")
print(f"      - Ingreso máximo por pedido: ${max(ingresos_pedidos):,.2f}")
print(f"   👤 Por Cliente:")
print(f"      - Total clientes únicos: {len(ingresos_totales_por_cliente):,}")
print(f"      - Ingreso promedio por cliente: ${np.mean(ingresos_clientes):.2f}")
print(f"      - Ingreso mediano por cliente: ${np.median(ingresos_clientes):.2f}")
print(f"   📊 General:")
print(f"      - Productos únicos con precios: {len(precio_promedio_por_producto):,}")
print(f"      - Ingreso total período validación: ${sum(ingresos_clientes):,.2f}")

Clientes únicos en validación: 25950
PREPARANDO DATOS DE INGRESOS PARA MODELO HÍBRIDO
ESTADÍSTICAS DE INGRESOS:
   📋 Por Pedido:
      - Total pedidos únicos: 37,306
      - Ingreso promedio por pedido: $79.48
      - Ingreso mediano por pedido: $23.83
      - Ingreso máximo por pedido: $38,013.84
   👤 Por Cliente:
      - Total clientes únicos: 25,950
      - Ingreso promedio por cliente: $114.26
      - Ingreso mediano por cliente: $35.46
   📊 General:
      - Productos únicos con precios: 3,184
      - Ingreso total período validación: $2,965,130.49


## Función para calcular métricas

In [107]:
def calcular_metricas_con_ingresos(recomendaciones_dict, compras_reales_cliente, cliente_id, k=5):
    """
    Calcula métricas para un cliente específico INCLUYENDO análisis de ingresos
    """
    metricas = {}
    
    for modelo, recs_df in recomendaciones_dict.items():
        # SALTEAR metadatos del diccionario
        if modelo in ['input_product', 'is_new_product']:
            continue
        
        # VALIDACIÓN ROBUSTA: Verificar que sea DataFrame válido
        if not isinstance(recs_df, pd.DataFrame):
            # Solo log para errores inesperados (no DataFrame)
            print(f"❌ Modelo {modelo}: Error inesperado, tipo: {type(recs_df)}")
            continue
            
        if recs_df.empty:
            # SILENCIOSO: Es normal que algunos modelos estén vacíos
            continue
            
        # VALIDACIÓN: Verificar que tenga columna 'producto'
        if 'producto' not in recs_df.columns:
            print(f"❌ Modelo {modelo}: Error estructura. Columnas: {recs_df.columns.tolist()}")
            continue
        
        try:
            # Obtener top K recomendaciones DE FORMA SEGURA
            productos_recomendados = set(recs_df.head(k)['producto'].tolist())
            productos_comprados = compras_reales_cliente
            
            # Calcular intersección (aciertos)
            hits_productos = productos_recomendados.intersection(productos_comprados)
            hits = len(hits_productos)
            
            # MÉTRICAS TRADICIONALES
            hit_rate = 1 if hits > 0 else 0
            
            if len(productos_recomendados) > len(productos_comprados):
                precision = hits / len(productos_comprados) 
            else:
                precision = hits / len(productos_recomendados) 
                
            recall = hits / len(productos_comprados) if productos_comprados else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            # NUEVAS: MÉTRICAS DE INGRESOS
            
            # Ingresos de productos acertados (lo que realmente ganamos por las recomendaciones)
            ingreso_acertado = 0
            for producto in hits_productos:
                if producto in ingresos_por_cliente_producto[cliente_id]:
                    ingreso_acertado += ingresos_por_cliente_producto[cliente_id][producto]
            
            # Ingreso total del cliente en validación
            ingreso_total_cliente = ingresos_totales_por_cliente[cliente_id]
            
            # Ingreso potencial si acepta todas las recomendaciones (estimación)
            ingreso_potencial_recomendaciones = 0
            for producto in productos_recomendados:
                if producto in precio_promedio_por_producto:
                    ingreso_potencial_recomendaciones += precio_promedio_por_producto[producto]
            
            # Tasa de captura de ingresos (qué % del gasto del cliente capturamos con recomendaciones)
            tasa_captura = (ingreso_acertado / ingreso_total_cliente) * 100 if ingreso_total_cliente > 0 else 0
            
            metricas[modelo] = {
                # Métricas tradicionales
                'hits': hits,
                'hit_rate': hit_rate,
                'precision': precision,
                'recall': recall,
                'f1_score': f1,
                'productos_recomendados': len(productos_recomendados),
                'productos_comprados': len(productos_comprados),
                
                # NUEVAS: Métricas de ingresos
                'ingreso_acertado': ingreso_acertado,
                'ingreso_total_cliente': ingreso_total_cliente,
                'ingreso_potencial_recomendaciones': ingreso_potencial_recomendaciones,
                'tasa_captura_ingresos': tasa_captura
            }
            
        except Exception as e:
            print(f"❌ Error procesando modelo {modelo}: {e}")
            continue
    
    return metricas

## Función principal de evaluación

In [108]:
def evaluar_algoritmo_con_ingresos(productos_test, n_productos_evaluar=100, k=10):
    """
    Evalúa el algoritmo con una muestra de productos INCLUYENDO análisis de ingresos
    """
    print(f"Evaluando {n_productos_evaluar} productos con top-{k} recomendaciones...")
    print("INCLUYENDO análisis de ingresos por cliente...")
    
    # Tomar muestra de productos para evaluar
    productos_evaluar = list(productos_test)[:n_productos_evaluar]
    
    # Diccionarios para acumular métricas tradicionales
    metricas_totales = defaultdict(list)
    
    # NUEVOS: Diccionarios para acumular métricas de ingresos
    metricas_ingresos = defaultdict(list)
    
    productos_evaluados = 0
    productos_sin_clientes = 0
    errores = 0
    
    for i, producto in enumerate(productos_evaluar):
        # Mostrar progreso cada 10%
        if i % max(1, len(productos_evaluar) // 10) == 0:
            print(f"Progreso: {i+1}/{len(productos_evaluar)} ({(i+1)/len(productos_evaluar)*100:.1f}%)")
        
        try:
            # Generar recomendaciones para este producto
            recomendaciones = test_all_models(producto, N=k)
            
            # Buscar clientes que compraron este producto en validación
            clientes_que_compraron = []
            for cliente, productos_comprados in compras_reales.items():
                if producto in productos_comprados:
                    clientes_que_compraron.append(cliente)
            
            if not clientes_que_compraron:
                productos_sin_clientes += 1
                continue
                
            # Evaluar para cada cliente que compró este producto
            for cliente in clientes_que_compraron:
                compras_cliente = compras_reales[cliente]
                
                # Calcular métricas tradicionales + ingresos
                metricas_cliente = calcular_metricas_con_ingresos(recomendaciones, compras_cliente, cliente, k)
                
                # Acumular métricas tradicionales
                for modelo, metricas in metricas_cliente.items():
                    for metrica, valor in metricas.items():
                        metricas_totales[f"{modelo}_{metrica}"].append(valor)
            
            productos_evaluados += 1
            
        except Exception as e:
            errores += 1
            if errores <= 5:
                print(f"Error evaluando producto {producto}: {e}")
            continue
    
    print(f"\nESTADÍSTICAS FINALES:")
    print(f"  Productos evaluados: {productos_evaluados}")
    print(f"  Sin clientes en validación: {productos_sin_clientes}")
    print(f"  Errores: {errores}")
    
    return metricas_totales, productos_evaluados

def calcular_metricas_promedio_con_ingresos(resultados):
    """
    Calcula métricas promedio por modelo INCLUYENDO análisis de ingresos
    """
    modelos = ['content', 'co_purchase', 'co_quotation', 'popularity', 'hybrid']
    metricas_finales = {}
    
    for modelo in modelos:
        if f"{modelo}_hit_rate" in resultados:
            # Métricas tradicionales
            metricas_tradicionales = {
                'hit_rate': np.mean(resultados[f"{modelo}_hit_rate"]),
                'precision': np.mean(resultados[f"{modelo}_precision"]),
                'recall': np.mean(resultados[f"{modelo}_recall"]),
                'f1_score': np.mean(resultados[f"{modelo}_f1_score"]),
                'n_evaluaciones': len(resultados[f"{modelo}_hit_rate"])
            }
            
            # NUEVAS: Métricas de ingresos (si existen)
            metricas_ingresos = {}
            if f"{modelo}_ingreso_acertado" in resultados:
                ingresos_acertados = resultados[f"{modelo}_ingreso_acertado"]
                ingresos_totales = resultados[f"{modelo}_ingreso_total_cliente"]
                ingresos_potenciales = resultados[f"{modelo}_ingreso_potencial_recomendaciones"]
                tasa_captura = resultados[f"{modelo}_tasa_captura_ingresos"]
                
                metricas_ingresos = {
                    'ingreso_promedio_acertado': np.mean(ingresos_acertados),
                    'ingreso_promedio_total': np.mean(ingresos_totales),
                    'ingreso_promedio_potencial': np.mean(ingresos_potenciales),
                    'tasa_captura_promedio': np.mean(tasa_captura),
                    'ingreso_total_acertado': sum(ingresos_acertados),
                    'ingreso_total_evaluado': sum(ingresos_totales),
                    'uplift_monetario_pct': (sum(ingresos_acertados) / sum(ingresos_totales)) * 100 if sum(ingresos_totales) > 0 else 0
                }
            
            # Combinar métricas
            metricas_finales[modelo] = {**metricas_tradicionales, **metricas_ingresos}
    
    return metricas_finales


## Ejecutar evaluación

In [109]:
# Ejecutar evaluación con ingresos
productos_test = set(test_data['producto'].unique())
productos_train = set(train_data['producto'].unique())
productos_comunes = productos_test.intersection(productos_train)

resultados_con_ingresos, n_evaluados = evaluar_algoritmo_con_ingresos(
    productos_comunes, 
    n_productos_evaluar=len(productos_comunes), 
    # n_productos_evaluar=10, 
    
    k=5
)

print(f"\nProductos evaluados exitosamente: {n_evaluados} / {len(productos_comunes)}")
print(f"Evaluaciones totales realizadas: {len(resultados_con_ingresos.get('content_hit_rate', []))}")

# Calcular métricas promedio con ingresos
metricas_finales_con_ingresos = calcular_metricas_promedio_con_ingresos(resultados_con_ingresos)

Evaluando 3085 productos con top-5 recomendaciones...
INCLUYENDO análisis de ingresos por cliente...
Progreso: 1/3085 (0.0%)
Progreso: 309/3085 (10.0%)
Progreso: 617/3085 (20.0%)
Progreso: 925/3085 (30.0%)
Progreso: 1233/3085 (40.0%)
Progreso: 1541/3085 (50.0%)
Progreso: 1849/3085 (59.9%)
Progreso: 2157/3085 (69.9%)
Progreso: 2465/3085 (79.9%)
Progreso: 2773/3085 (89.9%)
Progreso: 3081/3085 (99.9%)

ESTADÍSTICAS FINALES:
  Productos evaluados: 3085
  Sin clientes en validación: 0
  Errores: 0

Productos evaluados exitosamente: 3085 / 3085
Evaluaciones totales realizadas: 77738


In [110]:

# Convertir a DataFrame para visualización
df_metricas_con_ingresos = pd.DataFrame(metricas_finales_con_ingresos).T

# Redondear valores numéricos para mejor visualización
df_metricas_con_ingresos = df_metricas_con_ingresos.round(4)

df_metricas_con_ingresos

,hit_rate,precision,recall,f1_score,n_evaluaciones,ingreso_promedio_acertado,ingreso_promedio_total,ingreso_promedio_potencial,tasa_captura_promedio,ingreso_total_acertado,ingreso_total_evaluado,uplift_monetario_pct
content,0.0956,0.0243,0.0151,0.0175,77738.0,2.5519,279.8066,41.4839,0.9733,1.983822e+05,2.175160e+07,0.9120
co_purchase,0.4598,0.1476,0.1051,0.1171,77670.0,15.5746,279.5805,18.8234,6.1557,1.209680e+06,2.171501e+07,5.5707
co_quotation,0.4129,0.1316,0.0856,0.0975,22083.0,7.5353,268.2809,38.6767,4.7265,1.664031e+05,5.924448e+06,2.8088
popularity,0.1175,0.0321,0.0191,0.0223,77738.0,5.3896,279.8066,35.0255,1.4662,4.189790e+05,2.175160e+07,1.9262
hybrid,0.4519,0.1449,0.1034,0.1151,77738.0,14.7991,279.8066,18.9909,5.9157,1.150455e+06,2.175160e+07,5.2891


##  Evaluación por transacción multi-producto

In [111]:
def evaluar_ventas_cruzadas_por_transaccion(n_transacciones_evaluar=500, k=10):
    """
    Evalúa si el algoritmo puede predecir productos adicionales en transacciones multi-producto.
    Dado el primer producto de una transacción, ¿recomienda los otros productos comprados?
    INCLUYE ANÁLISIS DE INGRESOS POR TRANSACCIÓN
    """

    # 1. Encontrar transacciones multi-producto en datos de validación
    print("\n📊 Analizando transacciones multi-producto en período de validación...")
    
    # Agrupar por cliente y fecha para identificar transacciones
    test_data['fecha'] = pd.to_datetime(test_data['fecha'])
    transacciones_grupo = test_data.groupby(['id', 'fecha'])
    
    # Filtrar solo transacciones con más de 1 producto
    transacciones_multi = []
    for (cliente, fecha), grupo in transacciones_grupo:
        productos = grupo['producto'].unique()
        if len(productos) >= 2:  # Transacciones con 2+ productos
            transacciones_multi.append({
                'cliente': cliente,
                'fecha': fecha,
                'productos': list(productos),
                'n_productos': len(productos),
                'pedidos': grupo['pedido'].unique().tolist()  # Para análisis de ingresos
            })
    
    print(f"📈 Transacciones multi-producto encontradas: {len(transacciones_multi)}")
    
    if len(transacciones_multi) == 0:
        print("❌ No se encontraron transacciones multi-producto")
        return {}
    
    # Estadísticas de transacciones multi-producto
    tamaños = [t['n_productos'] for t in transacciones_multi]
    print(f"📊 Estadísticas de tamaño de transacciones:")
    print(f"   - Promedio productos por pedido: {np.mean(tamaños):.2f}")
    print(f"   - Mediana: {np.median(tamaños):.0f}")
    print(f"   - Máximo: {max(tamaños)}")
    print(f"   - Mínimo: {min(tamaños)}")
    
    # Distribución de tamaños
    from collections import Counter
    distribucion = Counter(tamaños)
    print(f"📋 Distribución de tamaños de pedido:")
    for tam in sorted(distribucion.keys())[:10]:  # Mostrar primeros 10
        print(f"   - {tam} productos: {distribucion[tam]} pedidos")
    if len(distribucion) > 10:
        print(f"   - ... y {len(distribucion) - 10} tamaños más")
    
    # 2. Tomar muestra de transacciones para evaluar
    n_evaluar = min(n_transacciones_evaluar, len(transacciones_multi))
    transacciones_evaluar = transacciones_multi[:n_evaluar]
    
    print(f"\n🎯 Evaluando {n_evaluar} transacciones...")
    print("Metodología: Dado el primer producto del pedido → ¿Se recomiendan los otros productos del mismo pedido?")
    print("💰 INCLUYENDO análisis de ingresos por transacción...")
    
    # 3. Evaluar cada transacción
    metricas_transacciones = defaultdict(list)
    metricas_ingresos_transacciones = defaultdict(list)  # NUEVO: para métricas de ingresos
    transacciones_evaluadas = 0
    errores = 0
    
    # Para debug: mostrar detalles de las primeras 3 transacciones
    mostrar_debug = True
    
    for i, transaccion in enumerate(transacciones_evaluar):
        # Mostrar progreso cada 10%
        if i % max(1, len(transacciones_evaluar) // 10) == 0:
            print(f"Progreso: {i+1}/{len(transacciones_evaluar)} ({(i+1)/len(transacciones_evaluar)*100:.1f}%)")
        
        try:
            productos = transaccion['productos']
            producto_input = productos[0]  # Primer producto como input
            productos_target = set(productos[1:])  # Resto como target
            pedidos_transaccion = transaccion['pedidos']
            
            # DEBUG: Mostrar detalles de las primeras 3 transacciones
            if mostrar_debug and transacciones_evaluadas < 3:
                print(f"\n" + "="*60)
                print(f"🔍 DEBUG TRANSACCIÓN {transacciones_evaluadas + 1}")
                print(f"📅 Cliente: {transaccion['cliente']}, Fecha: {transaccion['fecha'].strftime('%Y-%m-%d')}")
                print(f"🛒 Productos en transacción: {productos}")
                print(f"🎯 Input: '{producto_input}' → Target: {sorted(list(productos_target))}")
            
            # Generar recomendaciones para el primer producto
            recomendaciones = test_all_models(producto_input, N=k)
            
            if mostrar_debug and transacciones_evaluadas < 3:
                print(f"\n📋 Recomendaciones generadas para '{producto_input}':")
                for modelo, recs_df in recomendaciones.items():
                    if modelo not in ['input_product', 'is_new_product'] and not recs_df.empty:
                        productos_rec = recs_df['producto'].tolist()
                        print(f"  {modelo}: {productos_rec}")
                    elif modelo not in ['input_product', 'is_new_product']:
                        print(f"  {modelo}: Sin recomendaciones")
            
            # NUEVO: Calcular ingresos de la transacción
            ingreso_total_transaccion = 0
            ingresos_por_producto_transaccion = {}
            
            for pedido_id in pedidos_transaccion:
                if pedido_id in ingresos_totales_por_pedido:
                    ingreso_total_transaccion += ingresos_totales_por_pedido[pedido_id]
                    
                if pedido_id in ingresos_por_pedido_producto:
                    for prod, ingreso in ingresos_por_pedido_producto[pedido_id].items():
                        if prod in ingresos_por_producto_transaccion:
                            ingresos_por_producto_transaccion[prod] += ingreso
                        else:
                            ingresos_por_producto_transaccion[prod] = ingreso
            
            # Evaluar cada modelo
            for modelo, recs_df in recomendaciones.items():
                if modelo in ['input_product', 'is_new_product'] or recs_df.empty:
                    continue
                
                # Validar que sea DataFrame con columna 'producto'
                if not isinstance(recs_df, pd.DataFrame) or 'producto' not in recs_df.columns:
                    continue
                
                # Productos recomendados (top-K)
                productos_recomendados = set(recs_df.head(k)['producto'].tolist())
                
                # Calcular intersección
                productos_acertados = productos_recomendados.intersection(productos_target)
                
                # MÉTRICAS TRADICIONALES
                hit_rate = 1 if len(productos_acertados) > 0 else 0
                precision = len(productos_acertados) / len(productos_recomendados) if productos_recomendados else 0
                
                if len(productos_recomendados) > len(productos_target):
                    precision = len(productos_acertados) / len(productos_target) 
                else:
                    precision = len(productos_acertados) / len(productos_recomendados) 
                
                recall = len(productos_acertados) / len(productos_target) if productos_target else 0
                f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
                
                # NUEVAS: MÉTRICAS DE INGRESOS
                # Ingresos de productos acertados en esta transacción
                ingreso_acertado_transaccion = 0
                for producto in productos_acertados:
                    if producto in ingresos_por_producto_transaccion:
                        ingreso_acertado_transaccion += ingresos_por_producto_transaccion[producto]
                
                # Ingreso potencial si acepta todas las recomendaciones
                ingreso_potencial_recomendaciones = 0
                for producto in productos_recomendados:
                    if producto in precio_promedio_por_producto:
                        ingreso_potencial_recomendaciones += precio_promedio_por_producto[producto]
                
                # Tasa de captura de ingresos para esta transacción
                tasa_captura_transaccion = (ingreso_acertado_transaccion / ingreso_total_transaccion) * 100 if ingreso_total_transaccion > 0 else 0
                
                # Guardar métricas tradicionales
                metricas_transacciones[f"{modelo}_hit_rate"].append(hit_rate)
                metricas_transacciones[f"{modelo}_precision"].append(precision)
                metricas_transacciones[f"{modelo}_recall"].append(recall)
                metricas_transacciones[f"{modelo}_f1_score"].append(f1_score)
                metricas_transacciones[f"{modelo}_productos_acertados"].append(len(productos_acertados))
                metricas_transacciones[f"{modelo}_productos_target"].append(len(productos_target))
                
                # NUEVO: Guardar métricas de ingresos
                metricas_ingresos_transacciones[f"{modelo}_ingreso_acertado"].append(ingreso_acertado_transaccion)
                metricas_ingresos_transacciones[f"{modelo}_ingreso_total_transaccion"].append(ingreso_total_transaccion)
                metricas_ingresos_transacciones[f"{modelo}_ingreso_potencial_recomendaciones"].append(ingreso_potencial_recomendaciones)
                metricas_ingresos_transacciones[f"{modelo}_tasa_captura_transaccion"].append(tasa_captura_transaccion)
                
                # DEBUG: Mostrar métricas detalladas
                if mostrar_debug and transacciones_evaluadas < 3:
                    print(f"\n  🔹 {modelo.upper()}:")
                    print(f"    Recomendados: {sorted(list(productos_recomendados))}")
                    print(f"    Target:       {sorted(list(productos_target))}")
                    print(f"    ✅ Acertados: {sorted(list(productos_acertados))}")
                    print(f"    Hit Rate:  {hit_rate}")
                    print(f"    Precision: {precision:.3f}")
                    print(f"    Recall:    {recall:.3f}")
                    print(f"    F1-Score:  {f1_score:.3f}")
                    print(f"    💰 Ingreso total transacción: ${ingreso_total_transaccion:.2f}")
                    print(f"    💰 Ingreso acertado: ${ingreso_acertado_transaccion:.2f}")
                    print(f"    💰 Tasa captura: {tasa_captura_transaccion:.2f}%")
            
            if mostrar_debug and transacciones_evaluadas < 3:
                print("="*60)
            
            transacciones_evaluadas += 1
            
        except Exception as e:
            errores += 1
            if errores <= 5:
                print(f"Error evaluando transacción {i}: {e}")
            continue
    
    print(f"\n📈 ESTADÍSTICAS FINALES:")
    print(f"  ✅ Transacciones evaluadas: {transacciones_evaluadas}")
    print(f"  ❌ Errores: {errores}")
    
    # 4. Calcular métricas promedio (tradicionales + ingresos)
    print(f"\n📊 RESULTADOS DE VENTAS CRUZADAS POR TRANSACCIÓN:")
    print("="*60)
    
    # ACTUALIZADO: Incluir modelo de popularidad
    modelos = ['content', 'co_purchase', 'co_quotation', 'popularity', 'hybrid']
    metricas_finales = {}
    
    for modelo in modelos:
        if f"{modelo}_hit_rate" in metricas_transacciones:
            # Métricas tradicionales
            hit_rate_avg = np.mean(metricas_transacciones[f"{modelo}_hit_rate"])
            precision_avg = np.mean(metricas_transacciones[f"{modelo}_precision"])
            recall_avg = np.mean(metricas_transacciones[f"{modelo}_recall"])
            f1_avg = np.mean(metricas_transacciones[f"{modelo}_f1_score"])
            
            # NUEVAS: Métricas de ingresos
            ingresos_acertados = metricas_ingresos_transacciones[f"{modelo}_ingreso_acertado"]
            ingresos_totales = metricas_ingresos_transacciones[f"{modelo}_ingreso_total_transaccion"]
            ingresos_potenciales = metricas_ingresos_transacciones[f"{modelo}_ingreso_potencial_recomendaciones"]
            tasa_captura = metricas_ingresos_transacciones[f"{modelo}_tasa_captura_transaccion"]
            
            metricas_finales[modelo] = {
                # Métricas tradicionales
                'hit_rate': hit_rate_avg,
                'precision': precision_avg,
                'recall': recall_avg,
                'f1_score': f1_avg,
                'n_transacciones': len(metricas_transacciones[f"{modelo}_hit_rate"]),
                
                # NUEVAS: Métricas de ingresos
                'ingreso_promedio_acertado': np.mean(ingresos_acertados),
                'ingreso_promedio_total': np.mean(ingresos_totales),
                'ingreso_promedio_potencial': np.mean(ingresos_potenciales),
                'tasa_captura_promedio': np.mean(tasa_captura),
                'ingreso_total_acertado': sum(ingresos_acertados),
                'ingreso_total_evaluado': sum(ingresos_totales),
                'uplift_monetario_pct': (sum(ingresos_acertados) / sum(ingresos_totales)) * 100 if sum(ingresos_totales) > 0 else 0
            }
            
            print(f"\n🔹 {modelo.upper()}:")
            print(f"  Hit Rate@{k}:     {hit_rate_avg:.3f} ({hit_rate_avg*100:.1f}% de pedidos con al menos 1 acierto)")
            print(f"  Precision@{k}:    {precision_avg:.3f}")
            print(f"  Recall@{k}:       {recall_avg:.3f}")
            print(f"  F1-Score:         {f1_avg:.3f}")
            print(f"  Pedidos evaluados: {len(metricas_transacciones[f'{modelo}_hit_rate'])}")
            print(f"  💰 Ingreso promedio acertado: ${np.mean(ingresos_acertados):.2f}")
            print(f"  💰 Uplift monetario: {(sum(ingresos_acertados) / sum(ingresos_totales)) * 100 if sum(ingresos_totales) > 0 else 0:.2f}%")
    
    return metricas_finales, metricas_transacciones, metricas_ingresos_transacciones

In [112]:
# EJECUTAR EVALUACIÓN DE VENTAS CRUZADAS CON INGRESOS

resultados_ventas_cruzadas, metricas_detalladas, metricas_ingresos_detalladas = evaluar_ventas_cruzadas_por_transaccion(
    n_transacciones_evaluar=9999999, 
    k=10
)



📊 Analizando transacciones multi-producto en período de validación...
📈 Transacciones multi-producto encontradas: 16872
📊 Estadísticas de tamaño de transacciones:
   - Promedio productos por pedido: 3.89
   - Mediana: 3
   - Máximo: 34
   - Mínimo: 2
📋 Distribución de tamaños de pedido:
   - 2 productos: 6382 pedidos
   - 3 productos: 4207 pedidos
   - 4 productos: 2067 pedidos
   - 5 productos: 1248 pedidos
   - 6 productos: 849 pedidos
   - 7 productos: 549 pedidos
   - 8 productos: 401 pedidos
   - 9 productos: 321 pedidos
   - 10 productos: 229 pedidos
   - 11 productos: 173 pedidos
   - ... y 21 tamaños más

🎯 Evaluando 16872 transacciones...
Metodología: Dado el primer producto del pedido → ¿Se recomiendan los otros productos del mismo pedido?
💰 INCLUYENDO análisis de ingresos por transacción...
Progreso: 1/16872 (0.0%)

🔍 DEBUG TRANSACCIÓN 1
📅 Cliente: 198, Fecha: 1973-01-16
🛒 Productos en transacción: ['producto_3780', 'producto_162']
🎯 Input: 'producto_3780' → Target: ['produ

In [113]:
# Mostrar resultados en un DataFrame de forma legible
if resultados_ventas_cruzadas:
    resultados_df = pd.DataFrame(resultados_ventas_cruzadas).T
    print("\n📊 Resultados de métricas de ventas cruzadas CON INGRESOS:")
    display(resultados_df)


📊 Resultados de métricas de ventas cruzadas CON INGRESOS:


,hit_rate,precision,recall,f1_score,n_transacciones,ingreso_promedio_acertado,ingreso_promedio_total,ingreso_promedio_potencial,tasa_captura_promedio,ingreso_total_acertado,ingreso_total_evaluado,uplift_monetario_pct
content,0.075892,0.036837,0.036691,0.036749,16853.0,1.633111,142.868478,84.281686,1.819214,27522.8278,2.407762e+06,1.143087
co_purchase,0.494829,0.271707,0.270419,0.270923,16824.0,15.552314,142.869309,48.275443,13.200803,261652.1291,2.403633e+06,10.885693
co_quotation,0.389329,0.199846,0.198451,0.198986,6054.0,6.393480,136.434226,78.962896,8.748910,38706.1300,8.259728e+05,4.686126
popularity,0.107399,0.055595,0.055444,0.055505,16853.0,3.622653,142.868478,71.327808,2.835278,61052.5688,2.407762e+06,2.535656
hybrid,0.487391,0.267865,0.266610,0.267101,16853.0,14.832707,142.868478,48.484237,12.915067,249975.6156,2.407762e+06,10.382071


In [114]:
def evaluar_ventas_cruzadas_por_transaccion_multiproducto(n_transacciones_evaluar=500, k=5):
    """
    Evalúa si el algoritmo puede predecir productos adicionales en transacciones multi-producto.
    Dado el primer producto de una transacción, ¿recomienda los otros productos comprados?
    INCLUYE ANÁLISIS DE INGRESOS POR PEDIDO
    """

    # 1. Encontrar transacciones multi-producto usando la columna 'pedido'
    print("\n Analizando transacciones multi-producto en período de validación...")
    
    # Agrupar por pedido para identificar transacciones
    transacciones_grupo = test_data.groupby('pedido')
    
    # Filtrar solo transacciones con más de 1 producto
    transacciones_multi = []
    for pedido_id, grupo in transacciones_grupo:
        productos = grupo['producto'].unique()
        if len(productos) >= 2:  # Transacciones con 2+ productos
            transacciones_multi.append({
                'pedido_id': pedido_id,
                'cliente': grupo['id'].iloc[0],  # Cliente del pedido
                'fecha': grupo['fecha'].iloc[0] if 'fecha' in grupo.columns else 'N/A',
                'productos': list(productos),
                'n_productos': len(productos)
            })
    
    print(f"📈 Transacciones multi-producto encontradas: {len(transacciones_multi)}")
    
    if len(transacciones_multi) == 0:
        print("❌ No se encontraron transacciones multi-producto")
        return {}
    
    # Estadísticas de transacciones multi-producto
    tamaños = [t['n_productos'] for t in transacciones_multi]
    print(f"📊 Estadísticas de tamaño de transacciones:")
    print(f"   - Promedio productos por pedido: {np.mean(tamaños):.2f}")
    print(f"   - Mediana: {np.median(tamaños):.0f}")
    print(f"   - Máximo: {max(tamaños)}")
    print(f"   - Mínimo: {min(tamaños)}")
    

    # 2. Tomar muestra de transacciones para evaluar
    n_evaluar = min(n_transacciones_evaluar, len(transacciones_multi))
    transacciones_evaluar = transacciones_multi[:n_evaluar]
    
    print(f"\n🎯 Evaluando {n_evaluar} transacciones...")

    # 3. Evaluar cada transacción
    metricas_transacciones = defaultdict(list)
    metricas_ingresos_pedidos = defaultdict(list)  # NUEVO: para métricas de ingresos
    transacciones_evaluadas = 0
    errores = 0
    
    # Para debug: mostrar detalles de las primeras 3 transacciones
    mostrar_debug = True
    
    for i, transaccion in enumerate(transacciones_evaluar):
        # Mostrar progreso cada 10%
        if i % max(1, len(transacciones_evaluar) // 10) == 0:
            print(f"Progreso: {i+1}/{len(transacciones_evaluar)} ({(i+1)/len(transacciones_evaluar)*100:.1f}%)")
        
        try:
            productos = transaccion['productos']
            producto_input = productos[0]  # Primer producto como input
            productos_target = set(productos[1:])  # Resto como target
            pedido_id = transaccion['pedido_id']
            
            # DEBUG: Mostrar detalles de las primeras 3 transacciones
            # if mostrar_debug and transacciones_evaluadas < 3:
            #     print(f"\n" + "="*60)
            #     print(f"🔍 DEBUG TRANSACCIÓN {transacciones_evaluadas + 1}")
            #     print(f"🆔 Pedido: {transaccion['pedido_id']}")
            #     print(f"👤 Cliente: {transaccion['cliente']}")
            #     print(f"📅 Fecha: {transaccion['fecha']}")
            #     print(f"🛒 Productos en pedido: {productos}")
            #     print(f"🎯 Input: '{producto_input}' → Target: {sorted(list(productos_target))}")
            
            # Generar recomendaciones para el primer producto
            recomendaciones = test_all_models(producto_input, N=k)
            
            # if mostrar_debug and transacciones_evaluadas < 3:
            #     print(f"\n📋 Recomendaciones generadas para '{producto_input}':")
            #     for modelo, recs_df in recomendaciones.items():
            #         if modelo not in ['input_product', 'is_new_product'] and not recs_df.empty:
            #             productos_rec = recs_df['producto'].tolist()
            #             print(f"  {modelo}: {productos_rec}")
            #         elif modelo not in ['input_product', 'is_new_product']:
            #             print(f"  {modelo}: Sin recomendaciones")
            
            # NUEVO: Calcular ingresos del pedido
            ingreso_total_pedido = ingresos_totales_por_pedido.get(pedido_id, 0)
            ingresos_por_producto_pedido = ingresos_por_pedido_producto.get(pedido_id, {})
            
            # Evaluar cada modelo
            for modelo, recs_df in recomendaciones.items():
                if modelo in ['input_product', 'is_new_product'] or recs_df.empty:
                    continue
                
                # Validar que sea DataFrame con columna 'producto'
                if not isinstance(recs_df, pd.DataFrame) or 'producto' not in recs_df.columns:
                    continue
                
                # Productos recomendados (top-K)
                productos_recomendados = set(recs_df.head(k)['producto'].tolist())
                
                # Calcular intersección
                productos_acertados = productos_recomendados.intersection(productos_target)
                
                # MÉTRICAS TRADICIONALES
                hit_rate = 1 if len(productos_acertados) > 0 else 0
                
                if len(productos_recomendados) > len(productos_target):
                    precision = len(productos_acertados) / len(productos_target) 
                else:
                    precision = len(productos_acertados) / len(productos_recomendados) 
                
                recall = len(productos_acertados) / len(productos_target) if productos_target else 0
                f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
                
                # NUEVAS: MÉTRICAS DE INGRESOS
                # Ingresos de productos acertados en este pedido
                ingreso_acertado_pedido = 0
                for producto in productos_acertados:
                    if producto in ingresos_por_producto_pedido:
                        ingreso_acertado_pedido += ingresos_por_producto_pedido[producto]
                
                # Ingreso potencial si acepta todas las recomendaciones
                ingreso_potencial_recomendaciones = 0
                for producto in productos_recomendados:
                    if producto in precio_promedio_por_producto:
                        ingreso_potencial_recomendaciones += precio_promedio_por_producto[producto]
                
                # Tasa de captura de ingresos para este pedido
                tasa_captura_pedido = (ingreso_acertado_pedido / ingreso_total_pedido) * 100 if ingreso_total_pedido > 0 else 0
                
                # Guardar métricas tradicionales
                metricas_transacciones[f"{modelo}_hit_rate"].append(hit_rate)
                metricas_transacciones[f"{modelo}_precision"].append(precision)
                metricas_transacciones[f"{modelo}_recall"].append(recall)
                metricas_transacciones[f"{modelo}_f1_score"].append(f1_score)
                metricas_transacciones[f"{modelo}_productos_acertados"].append(len(productos_acertados))
                metricas_transacciones[f"{modelo}_productos_target"].append(len(productos_target))
                
                # NUEVO: Guardar métricas de ingresos
                metricas_ingresos_pedidos[f"{modelo}_ingreso_acertado"].append(ingreso_acertado_pedido)
                metricas_ingresos_pedidos[f"{modelo}_ingreso_total_pedido"].append(ingreso_total_pedido)
                metricas_ingresos_pedidos[f"{modelo}_ingreso_potencial_recomendaciones"].append(ingreso_potencial_recomendaciones)
                metricas_ingresos_pedidos[f"{modelo}_tasa_captura_pedido"].append(tasa_captura_pedido)
                
                # # DEBUG: Mostrar métricas detalladas
                # if mostrar_debug and transacciones_evaluadas < 3:
                #     print(f"\n  🔹 {modelo.upper()}:")
                #     print(f"    Recomendados: {sorted(list(productos_recomendados))}")
                #     print(f"    Target:       {sorted(list(productos_target))}")
                #     print(f"    ✅ Acertados: {sorted(list(productos_acertados))}")
                #     print(f"    Hit Rate:  {hit_rate}")
                #     print(f"    Precision: {precision:.3f}")
                #     print(f"    Recall:    {recall:.3f}")
                #     print(f"    F1-Score:  {f1_score:.3f}")
                #     print(f"    💰 Ingreso total pedido: ${ingreso_total_pedido:.2f}")
                #     print(f"    💰 Ingreso acertado: ${ingreso_acertado_pedido:.2f}")
                #     print(f"    💰 Tasa captura: {tasa_captura_pedido:.2f}%")
            
            if mostrar_debug and transacciones_evaluadas < 3:
                print("="*60)
            
            transacciones_evaluadas += 1
            
        except Exception as e:
            errores += 1
            if errores <= 5:
                print(f"Error evaluando transacción {i}: {e}")
            continue
    

    
    modelos = ['content', 'co_purchase', 'co_quotation', 'popularity', 'hybrid']
    metricas_finales = {}
    
    for modelo in modelos:
        if f"{modelo}_hit_rate" in metricas_transacciones:
            # Métricas tradicionales
            hit_rate_avg = np.mean(metricas_transacciones[f"{modelo}_hit_rate"])
            precision_avg = np.mean(metricas_transacciones[f"{modelo}_precision"])
            recall_avg = np.mean(metricas_transacciones[f"{modelo}_recall"])
            f1_avg = np.mean(metricas_transacciones[f"{modelo}_f1_score"])
            
            # NUEVAS: Métricas de ingresos
            ingresos_acertados = metricas_ingresos_pedidos[f"{modelo}_ingreso_acertado"]
            ingresos_totales = metricas_ingresos_pedidos[f"{modelo}_ingreso_total_pedido"]
            ingresos_potenciales = metricas_ingresos_pedidos[f"{modelo}_ingreso_potencial_recomendaciones"]
            tasa_captura = metricas_ingresos_pedidos[f"{modelo}_tasa_captura_pedido"]
            
            metricas_finales[modelo] = {
                # Métricas tradicionales
                'hit_rate': hit_rate_avg,
                'precision': precision_avg,
                'recall': recall_avg,
                'f1_score': f1_avg,
                'n_transacciones': len(metricas_transacciones[f"{modelo}_hit_rate"]),
                
                # NUEVAS: Métricas de ingresos
                'ingreso_promedio_acertado': np.mean(ingresos_acertados),
                'ingreso_promedio_total': np.mean(ingresos_totales),
                'ingreso_promedio_potencial': np.mean(ingresos_potenciales),
                'tasa_captura_promedio': np.mean(tasa_captura),
                'ingreso_total_acertado': sum(ingresos_acertados),
                'ingreso_total_evaluado': sum(ingresos_totales),
                'uplift_monetario_pct': (sum(ingresos_acertados) / sum(ingresos_totales)) * 100 if sum(ingresos_totales) > 0 else 0
            }
            
            print(f"\n🔹 {modelo.upper()}:")
            print(f"  Hit Rate@{k}:     {hit_rate_avg:.3f} ({hit_rate_avg*100:.1f}% de pedidos con al menos 1 acierto)")
            print(f"  Precision@{k}:    {precision_avg:.3f}")
            print(f"  Recall@{k}:       {recall_avg:.3f}")
            print(f"  F1-Score:         {f1_avg:.3f}")
            print(f"  Pedidos evaluados: {len(metricas_transacciones[f'{modelo}_hit_rate'])}")
            print(f"   Ingreso promedio acertado: ${np.mean(ingresos_acertados):.2f}")
            print(f"   Uplift monetario: {(sum(ingresos_acertados) / sum(ingresos_totales)) * 100 if sum(ingresos_totales) > 0 else 0:.2f}%")
    
    return metricas_finales, metricas_transacciones, metricas_ingresos_pedidos

In [ ]:
# CONTAR TODAS LAS TRANSACCIONES MULTI-PRODUCTO
print("Contando todas las transacciones multi-producto en datos de test...")

transacciones_grupo = test_data.groupby('pedido')
# Seleccionar un subconjunto pequeño de transacciones_grupo para pruebas rápidas

transacciones_multi_total = 0

for pedido_id, grupo in transacciones_grupo:
    productos = grupo['producto'].unique()
    if len(productos) >= 2:
        transacciones_multi_total += 1

print(f"Total de pedidos multi-producto a evaluar: {transacciones_multi_total}")

# EJECUTAR CON TODOS LOS DATOS CON INGRESOS
print("\nINICIANDO EVALUACIÓN DE VENTAS CRUZADAS CON TODOS LOS PEDIDOS MULTI-PRODUCTO...")
print("INCLUYENDO análisis de ingresos por pedido...")
print("Esto puede tomar mucho tiempo dependiendo del número de pedidos...")

resultados_ventas_cruzadas_completo, metricas_detalladas_completo, metricas_ingresos_completo = evaluar_ventas_cruzadas_por_transaccion_multiproducto(
    n_transacciones_evaluar=transacciones_multi_total,  # TODOS los pedidos multi-producto
    k=10
)


Contando todas las transacciones multi-producto en datos de test...
Total de pedidos multi-producto a evaluar: 16803

INICIANDO EVALUACIÓN DE VENTAS CRUZADAS CON TODOS LOS PEDIDOS MULTI-PRODUCTO...
INCLUYENDO análisis de ingresos por pedido...
Esto puede tomar mucho tiempo dependiendo del número de pedidos...

 Analizando transacciones multi-producto en período de validación...
📈 Transacciones multi-producto encontradas: 16803
📊 Estadísticas de tamaño de transacciones:
   - Promedio productos por pedido: 3.74
   - Mediana: 3
   - Máximo: 34
   - Mínimo: 2

🎯 Evaluando 16803 transacciones...
Progreso: 1/16803 (0.0%)
Error: Producto 'producto_5964' es nuevo pero no se proporcionaron características
Error: Producto 'producto_5970' es nuevo pero no se proporcionaron características
Error: Producto 'producto_5978' es nuevo pero no se proporcionaron características
Error: Producto 'producto_5981' es nuevo pero no se proporcionaron características
Error: Producto 'producto_5970' es nuevo pero

In [1]:
# Mostrar resultados completos CON INGRESOS
if resultados_ventas_cruzadas_completo:
    resultados_df_completo = pd.DataFrame(resultados_ventas_cruzadas_completo).T
    print("\n📊 Resultados completos de ventas cruzadas CON MÉTRICAS DE INGRESOS:")
    display(resultados_df_completo)
    
    # Guardar resultados completos con ingresos
    output_dir = '../Datos/validacion_historica'
    os.makedirs(output_dir, exist_ok=True)
    resultados_df_completo.to_csv(f'{output_dir}/metricas_ventas_cruzadas_pedidos_con_ingresos.csv')
    print(f"\n💾 Resultados completos con ingresos guardados en: {output_dir}/metricas_ventas_cruzadas_pedidos_con_ingresos.csv")
    
    # Mostrar resumen de mejores métricas de ingresos
    print(f"\n💰 RESUMEN DE MÉTRICAS DE INGRESOS:")
    print("="*50)
    for modelo in ['content', 'co_purchase', 'co_quotation', 'hybrid']:
        if modelo in resultados_df_completo.index:
            uplift = resultados_df_completo.loc[modelo, 'uplift_monetario_pct']
            ingreso_acertado = resultados_df_completo.loc[modelo, 'ingreso_total_acertado']
            print(f"🔹 {modelo.upper()}: Uplift {uplift:.2f}% | Ingreso capturado: ${ingreso_acertado:,.2f}")

NameError: name 'resultados_ventas_cruzadas_completo' is not defined